## Прогноз оклада по описанию вакансии
На основе данных, содержащих описания вакансий и соответствующие им годовые зарплаты, построить модель линейной регрессии с L2-регуляризацией и спрогнозировать оклады по описанию вакансий.

In [1]:
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

### Прочитаем исходные данные

In [2]:
data= pd.read_csv('salary-train.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
FullDescription       60000 non-null object
LocationNormalized    60000 non-null object
ContractTime          44418 non-null object
SalaryNormalized      60000 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [3]:
data.agg('nunique')

FullDescription       59821
LocationNormalized     1763
ContractTime              2
SalaryNormalized       3930
dtype: int64

In [4]:
data[data.duplicated()]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
49196,Appoint Group Vacancy Job Title Senior Nurse ...,Bury,NaN,40000


### В нашем распоряжении таблица из 60000 строк и 4 столбцов. В столбце "ContractTime" есть пропуски. В таблице есть дублирующаяся строка - удалим ее.

In [5]:
data=data.drop_duplicates()
data.shape

(59999, 4)

### Для дальнейшего разделения текста описания вакансий на слова:
- приведем все буквы к нижнему регистру,
- заменим все, кроме букв и цифр, на пробелы.

In [6]:
data['FullDescription']=[x.lower() for x in data.FullDescription]
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [7]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


### Преобразуем тексты описания вакансий в векторы признаков - оставим только те слова, которые встречаются хотя бы в 5 объектах.

In [8]:
vectorizer = TfidfVectorizer(min_df=5)
doc_term_mx=vectorizer.fit_transform(data.FullDescription)

In [9]:
len(vectorizer.get_feature_names())

22861

In [10]:
doc_term_mx.shape

(59999, 22861)

### Получили матрицу TF-IDF признаков для 22861 слова

### Заменим пропуски в столбце 'ContractTime' на nan

In [11]:
#data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


### Воспользуемся one-hot-кодированием, чтобы перекодировать признаки LocationNormalized и ContractTime

In [12]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [13]:
enc.get_feature_names()[:10]

['ContractTime=contract',
 'ContractTime=nan',
 'ContractTime=permanent',
 'LocationNormalized=Abbots Langley',
 'LocationNormalized=Aberaeron',
 'LocationNormalized=Aberdare',
 'LocationNormalized=Aberdeen',
 'LocationNormalized=Aberdeenshire',
 'LocationNormalized=Abergele',
 'LocationNormalized=Abertillery']

In [14]:
X_train_categ.shape

(59999, 1766)

### Объединим все полученные признаки в одну матрицу "объекты-признаки". 

In [15]:
data2=hstack([doc_term_mx,X_train_categ])

In [16]:
data2.shape

(59999, 24627)

### Обучим гребневую регрессию с целевой переменной в столбце SalaryNormalized

In [17]:
clf = Ridge(alpha=1.0, random_state=241)
clf.fit(data2, data.SalaryNormalized)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=241, solver='auto', tol=0.001)

### Прочитаем файл с примерами описания вакансий

In [18]:
data_test= pd.read_csv('salary-test-mini.csv')

In [19]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


### Проведем обработку данных, аналогичную той, что была сделана для тренировочных данных:
- приведем все буквы к нижнему регистру,
- заменим все, кроме букв и цифр, на пробелы,
- преобразуем тексты описания вакансий в векторы признаков - оставим только те слова, которые встречаются хотя бы в 5 объектах,
- воспользуемся one-hot-кодированием, чтобы перекодировать признаки LocationNormalized и ContractTime,
- объединим все полученные признаки в одну матрицу "объекты-признаки".

In [20]:
data_test['FullDescription']=[x.lower() for x in data_test.FullDescription]
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [21]:
data_test1=data_test
doc_term_mx_test=vectorizer.transform(data_test1.FullDescription)

In [22]:
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [23]:
data_test2=hstack([doc_term_mx_test,X_test_categ])

In [24]:
data_test2.shape

(2, 24627)

### Построим прогнозы для предложенных описаний вакансий

In [25]:
pred=clf.predict(data_test2)
print('Для вакансии № 0 в городе',data_test.LocationNormalized[0],'прогнозируемый оклад составляет',int(pred[0]),'y.e.')
print('Для вакансии № 1 в городе',data_test.LocationNormalized[1],'прогнозируемый оклад составляет',int(pred[1]),'y.e.')

Для вакансии № 0 в городе Milton Keynes прогнозируемый оклад составляет 56573 y.e.
Для вакансии № 1 в городе Manchester прогнозируемый оклад составляет 37199 y.e.
